In [1]:
import numpy as np
import pandas as pd
import os

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
print("UYARI ENGELLEYİCİ")

UYARI ENGELLEYİCİ


# VERİ

In [2]:
filepath = '/kaggle/input/commonlit-evaluate-student-summaries'

prompts_test = pd.read_csv(os.path.join(filepath, 'prompts_test.csv'))
prompts_train = pd.read_csv(os.path.join(filepath, 'prompts_train.csv'))
sub = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
summaries_test = pd.read_csv(os.path.join(filepath, 'summaries_test.csv'))
filtered_summaries_train = pd.read_csv(os.path.join(filepath, 'summaries_train.csv'))
print("DOSYA YOL KODLARI")

DOSYA YOL KODLARI


In [3]:
desired_length = 20
filtered_summaries_train = pd.concat([group.sample(desired_length) for _, group in filtered_summaries_train.groupby('prompt_id')])

In [4]:
import nltk
# NLTK'den tokenizer'ı yükle
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Tokenizasyon işlemi
filtered_summaries_train['tokenized_text'] = filtered_summaries_train['text'].apply(lambda x: word_tokenize(x))
prompts_test['prompt_text'] = prompts_test['prompt_text'].apply(lambda x: word_tokenize(x))
summaries_test['text'] = summaries_test['text'].apply(lambda x: word_tokenize(x))


[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


# Metin Temizliği

In [5]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

# NLTK stopwords verilerini indirin
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Temizleme işlemlerini uygulama fonksiyonu
def preprocess_text(tokenized_text):
    # Her bir kelimeyi temizle
    cleaned_words = []
    for word in tokenized_text:
        # Küçük harfe dönüştürme
        word = word.lower()
        # Noktalama işaretlerini boşlukla değiştirme
        word = re.sub(r'[^\w\s]', ' ', word)
        # Gereksiz boşlukları kaldırma
        word = re.sub(r'\s+', ' ', word)
        # Stopwords'leri kaldırma
        if word not in stop_words:
            cleaned_words.append(word)
    return ' '.join(cleaned_words)

# Tokenleştirilmiş metinleri temizleme
filtered_summaries_train['cleaned_text'] = filtered_summaries_train['tokenized_text'].apply(preprocess_text)
prompts_test['prompt_text'] = prompts_test['prompt_text'].apply(preprocess_text)
summaries_test['text'] = summaries_test['text'].apply(preprocess_text)

# Noktalama işaretlerini boşlukla değiştirme
filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('[^\w\s]', '')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('[^\w\s]', '')
summaries_test['text'] = summaries_test['text'].str.replace('[^\w\s]', '')

# ŞİMDİ BOŞLUKLARI KALDIRRRR HAYDİİİİİİİİİİİ
filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('\s+', '')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('\s+', '')
summaries_test['text'] = summaries_test['text'].str.replace('\s+', '')

#Bişey deniyorum
filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('  ', ' ')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('  ', ' ')
summaries_test['text'] = summaries_test['text'].str.replace('  ', ' ')

filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('   ', ' ')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('   ', ' ')
summaries_test['text'] = summaries_test['text'].str.replace('   ', ' ')

filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace('    ', ' ')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('    ', ' ')
summaries_test['text'] = summaries_test['text'].str.replace('    ', ' ')

# Temizlenmiş verileri gösterme
print(filtered_summaries_train['cleaned_text'])

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
3392    aristotle stated good tragedy simple plot one ...
5766    second rank comes kind tragedy place first  li...
3729    effect produced  aristotle argued catastrophe ...
2496    three elements ideal tragedy one issue  fortun...
4270    follow complex plan  imitate actions excite pi...
                              ...                        
3740    meat factory covered used spoiled meat using v...
5263    meat became sour rotten  companies found vario...
1899    factory would take meat  pickle would often fo...
6722    meat canning sausage piled floor workers carri...
225     overall  ways factory workers cover meat diffe...
Name: cleaned_text, Length: 80, dtype: object


# YANLIŞ KELİME DÜZELTME

In [6]:
# "text" sütunundaki tüm satırları birleştirip tek bir metin oluşturun
all_text = ' '.join(filtered_summaries_train['cleaned_text'])

# Metni boşluklardan bölerek kelimeleri ayırın ve bir liste oluşturun
words = all_text.split()

# Kelimelerin benzersiz (unique) olmasını sağlamak için bir küme (set) oluşturun
unique_words_list = list(set(words))

# Benzersiz kelimeleri görüntüleyin
print(len(unique_words_list))

1255


In [7]:
# 11. kelime ile 20. kelime arasındaki benzersiz kelimeleri görüntülemek için
print(unique_words_list[150:170])

['papyrus', 'billions', 'salesmen', 'suffer', 'sense', 'renowned', 'especially', 'close', 'hide', 'orestes', 'sometimes', 'stuffed', 'collected', 'hole', 'nobles', 'fo', 'enhance', 'pharaoh', 'principles', 'governemnt']


In [8]:
# Birden fazla kelimeyi düzeltmek için her bir düzeltmeyi ayrı satırlarda tanımlayın

corrections = {
    "structuree": "structure",
    "emploees": "employees",
    "governemtn": "government",
    "bellieved": "believed",
    "nuaseating":"nauseating",
    "extact":"extract",
    "valubule":"valuable",
    "reaso":"reason",
    "figting":"fighting",
    "complx":"complex",
    "sonstructed":"constructed",
    "differeny":"different",
    "instert":"insert",
    "experiemnt":"experiment",
    "folloeing":"following",
    "behaivior":"behavior",
    "beggng":"begging",
    "middl":"middle",
    "expirament":"experiment",
    "shoild":"should",
    "supposted":"supposed",
    "desissions":"decisions",
    "movemnet":"movement",
    "controllled":"controlled",
    "gorvernemt":"government",
    "afer":"after",
    "succesfull":"successful",
    "pharoaohs":"pharaohs",
    "pharoras":"pharaohs",
    "quaility":"quality",
    "tradedy":"tragedy",
    "wokers":"workers",
    "exampl":"example",
    "classś":"class",
    "includs":"includes",
    "susage":"sausage",
    "mobilty":"mobility",
    "saddness":"sadness",
    "sinckair":"sinclair",
    "artile":"article",
    "craftsmem":"craftsmen",
    "charactor":"character",
    "thorught":"through",
    "aviod":"avoid",
    "betweem":"between",
    "endof":"end of",
    "perceft":"perfect",
    "glycerince":"glycerine",
    "ful":"full"
}

# Her düzeltmeyi sırayla uygulayın
for old_word, new_word in corrections.items():
    filtered_summaries_train['cleaned_text'] = filtered_summaries_train['cleaned_text'].str.replace(old_word, new_word)

for old_word, new_word in corrections.items():
    filtered_summaries_train['text'] = filtered_summaries_train['text'].str.replace(old_word, new_word)

# Sütunun güncellendiğini kontrol etmek için ilk 10 satırı görüntüleyebilirsiniz
print("Başarılı")

Başarılı


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [10]:
# Model ve Tokenizer'ı yükleyin
model_name = '/kaggle/input/debertav3xsmall/deberta-v3-xsmall'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # İki etiket olduğunu varsayalım

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/debertav3xsmall/deberta-v3-xsmall and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Eğitim verilerini yükleyin
X = filtered_summaries_train['cleaned_text'].values
y = filtered_summaries_train[['content', 'wording']].values

# Veriyi tokenize edin ve tensörlere çevirin
def tokenize_data(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=300,
            return_tensors="pt"
        )
        
        input_ids.append(encoded["input_ids"])
        attention_masks.append(encoded["attention_mask"])
    
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0), torch.tensor(labels)

input_ids, attention_masks, labels = tokenize_data(X, y)

In [12]:
# Eğitilmiş modelin ağırlıklarını yükleyin
model.load_state_dict(torch.load('/kaggle/input/kaydedilecekmi/my_model.pth'))

<All keys matched successfully>

In [13]:
from torch.utils.data import TensorDataset, DataLoader

# Yukarıda verilen örneğe benzer şekilde eğitim verilerinizi ve etiketlerinizi tensörlere çevirdiyseniz,
# TensorDataset ve DataLoader kullanarak bir veri yükleyici oluşturabilirsiniz.

dataset = TensorDataset(input_ids, attention_masks, labels)
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [14]:
# İlk olarak content'i tahmin et
model.eval()
predictions_content = []
true_labels_content = []

for batch in dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).tolist()
    true_labels_content.extend(labels[:, 0].tolist())  # Sadece sol taraftaki (y1) değeri alıyoruz
    predictions_content.extend(predicted_labels)
# Tahminlerinizi kullanarak işlem yapabilirsiniz
# Örneğin, predictions_content ve predictions_wording'i kullanarak sonuçları değerlendirebilirsiniz

In [15]:
# Şimdi wording'i tahmin et
model.eval()
predictions_wording = []
true_labels_wording = []

for batch in dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).tolist()
    true_labels_wording.extend(labels[:, 1].tolist())  # Sağ taraftaki (y2) değeri alıyoruz
    predictions_wording.extend(predicted_labels)

In [16]:
# Mean Squared Error (MSE) hesaplamak için true_labels_wording ve predictions_wording kullanabilirsiniz
from sklearn.metrics import mean_squared_error
mse_y2 = mean_squared_error(true_labels_content, predictions_content)
print(f"Mean Squared Error (MSE) for content: {mse_y2:.4f}")

Mean Squared Error (MSE) for content: 1.2099


In [17]:
# Mean Squared Error (MSE) hesaplamak için true_labels_wording ve predictions_wording kullanabilirsiniz
mse_y2 = mean_squared_error(true_labels_wording, predictions_wording)
print(f"Mean Squared Error (MSE) for wording: {mse_y2:.4f}")

Mean Squared Error (MSE) for wording: 1.2969


In [18]:
from sklearn.metrics import mean_squared_error

# Verileriniz
y1_true = true_labels_content  # y1 gerçek değerler
y2_true = true_labels_wording   # y2 gerçek değerler
y1_pred = predictions_content   # y1 tahmin değerler
y2_pred = predictions_wording   # y2 tahmin değerler

def compute_mcrmse(y1_true, y1_pred, y2_true, y2_pred):
    # Her iki sütun için RMSE hesaplamaları yapın
    y1_rmse = np.sqrt(mean_squared_error(y1_true, y1_pred))
    y2_rmse = np.sqrt(mean_squared_error(y2_true, y2_pred))
    
    # Ortalama RMSE hesaplayın
    mcrmse = (y1_rmse + y2_rmse) / 2.0
    
    return {
        "y1_rmse": y1_rmse,
        "y2_rmse": y2_rmse,
        "mcrmse": mcrmse,
    }

# MCRMSE sonuçlarını hesaplayın
mcrmse_result = compute_mcrmse(y1_true, y1_pred, y2_true, y2_pred)

# MCRMSE sonuçlarını yazdırın
print("RMSE for y1:", mcrmse_result["y1_rmse"])
print("RMSE for y2:", mcrmse_result["y2_rmse"])
print("MCRMSE:", mcrmse_result["mcrmse"])

RMSE for y1: 1.0999663975680807
RMSE for y2: 1.1387934048143502
MCRMSE: 1.1193799011912153


In [19]:
# Content ve wording tahminlerinin ortalamasını alın
average_content = np.mean(predictions_content)
average_wording = np.mean(predictions_wording)

# Mevcut "sub" dosyasına tahminleri ekleyin
sub['content'] = average_content
sub['wording'] = average_wording

# Güncellenmiş gönderim dosyasını kaydedin
sub.to_csv('submission.csv', index=False)

In [20]:
sub

,student_id,content,wording
0,000000ffffff,0.0,0.0
1,111111eeeeee,0.0,0.0
2,222222cccccc,0.0,0.0
3,333333dddddd,0.0,0.0
